In [1]:
import numpy as np
!pip install pyMCFSimplex
from pyMCFSimplex import *

  Using cached pyMCFSimplex-0.9.4.tar.gz (164 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pyMCFSimplex: filename=pyMCFSimplex-0.9.4-cp39-cp39-win_amd64.whl size=228564 sha256=36804927121a8e1445846da1ca2f362cd928d519d68dcca7b6cc20c380b8d5dc
  Stored in directory: c:\users\valeria\appdata\local\pip\cache\wheels\54\bb\ed\1b739362e77897cc730a11c43358a6f24c749c0669790dbb99
Successfully built pyMCFSimplex



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### ESTRAZIONE DEI DATI 
La prima funzione serve per estrarre i costi quadratici che si trovano nella diagonale di Q,
la seconda funzione serve per estrarre le seguenti quantità 
 - u,b,q
 - numero nodi
 - numero archi 


In [2]:
def matrice_Q(nome_file):
    vettore = []
    with open(nome_file, 'r') as file:
        dimensione = int(file.readline()) # la prima riga è la dimensione del vettore
        valori = file.readline().split() # dalla seconda riga 
        vettore = [float(valore) for valore in valori]
        
        
        #error check 
        if len(vettore) != dimensione:
            raise ValueError("Il numero di valori nel file non corrisponde alla dimensione specificata.")
    
    dimensione = len(vettore)
    matrice = np.zeros((dimensione, dimensione)) #creo matrice
    np.fill_diagonal(matrice, vettore)  #riempio la diagonale
    return matrice, vettore
   

nome_file = "1000/netgen-1000-1-1-a-a-s.qfc"
Q_d = matrice_Q(nome_file)[1]
Q = matrice_Q(nome_file)[0]

In [4]:
def leggi_file_dimacs(file_path):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_=[]
    to_=[]
    edges = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':
                    # Leggi il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # Inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # Leggi i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # Assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # Leggi l'arc e il suo costo
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # Ora leggiamo il costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, u, b, q, edges,from_,to_

# Esempio di utilizzo
file_path = '1000/netgen-1000-1-1-a-a-s.dmx'  # Sostituisci con il percorso del tuo file DIMACS
numero_nodi, numero_archi, u, b, q, edges,from_ ,to_ = leggi_file_dimacs(file_path)

# creazione della matrice E 
E = np.zeros((numero_nodi, len(edges)), int)
#import pandas as pd
#E = pd.DataFrame(E)
#metto nomi delle colonne delle matrice E uguali agli archi
#E.columns = [edges]

### ALGORITMO 


In [20]:
#________________________
#      STEP 0 
#_______________________

import numpy as np
import random
from scipy.optimize import linprog
# Set a seed for the random number generator if needed
seed = 16  # You can use any integer value as the seed
np.random.seed(seed)
#c = np.zeros(len(u) + 1)
n = 1000
m = 89
c = np.zeros(n + 1)

# Define the equality constraint Ex = b
A_eq = np.column_stack((E, np.zeros(m)))  # Extend E with a dummy variable

# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b, bounds=list(zip(np.zeros(n), u)))

# Extract the solution vector x from the result
x = result.x[:-1]

print("Solution x:", x)




#________________________
#      STEP 1 
#_______________________

# calcolo del gradiente 

# gradient = 2 * np.dot(Q, x_scaled) + q
 
# calcolo delle x_bar 

#direzione 
#d= x_bar - x_0


#________________________
#      STEP 2 
#_______________________
#determinazione alpha, step size 
# alpha= 2/(2+k)
#alpha= 2/(2+0)


#________________________
#      STEP 3
#_______________________

# aggiornamento posizione 
#x_(k+1)=x_k + alpha_k *d_k


# check terminazione 

# k=k+1 

ValueError: Invalid input for linprog: unable to interpret bounds with this dimension tuple: (1000, 2).

In [41]:
x, residuals, _, _ = np.linalg.lstsq(E, b, rcond=None)

print("Solution x:")
print(x)

Solution x:
[-6.09148797e-03 -7.96409385e-03  7.29256976e-03 -1.20936830e-02
  3.64163688e-03  1.52523374e-03 -1.32884609e-02  1.12978055e-02
 -8.82865551e-04  1.23981273e-02 -1.05293662e-02 -1.34437527e-02
 -6.18642659e-03  1.14703834e-02 -1.17439158e-02  8.32813878e-03
  5.51825699e-03  2.12765666e-03  5.71003086e-04  7.69796963e-03
  1.82255117e-02 -3.01128320e-04 -1.72445467e-02 -4.01337995e-03
 -6.14283037e-03  2.36716561e-04  5.05935291e-03  9.31706612e-04
 -1.58122248e-02  6.23294179e-03 -8.61231127e-03  8.21822366e-03
 -1.55209439e-02  5.87457370e-03 -1.55134025e-02  4.54387241e-03
 -1.16476416e-03 -9.49334518e-03  5.01293852e-03  8.17552386e-03
  5.65621200e-03 -2.48095519e-02  1.08166366e-02  8.55688355e-03
 -1.44528971e-02 -2.03237070e-03  1.00299753e-02  1.23503639e-02
 -1.12286951e-02  3.72104213e-03 -6.13898001e-03  1.11966956e-02
  5.60820822e-03  6.29896210e-03 -1.38679870e-03  1.01136465e-02
 -1.23403650e-02 -1.76482900e-02 -5.04335630e-03 -1.90796620e-02
 -3.62028701e

In [49]:
def scale_vector(x, u):
    scaled_x = []
    for xi, ui in zip(x, u):
        scaling_factor = ui / max(abs(xi), 1e-6)
        scaled_xi = xi * scaling_factor
        scaled_x.append(scaled_xi)
    return scaled_x

scaled_x = scale_vector(x, u)

In [52]:
gradient = 2 * Q @ scaled_x + q

In [54]:
# MCFP problem transformed to integers and lists
nmx     = numero_nodi # max number of nodes
mmx     = numero_archi # max number of arcs
pn      = numero_nodi # current number of nodes
pm      = numero_archi# current number of arcs
pU      = u # column maxflow
pC      = gradient # column cost
pDfct   = b  # node deficit (supply/demand)
pSn     = from_ # column from
pEn     = to_ # column to

# call LoadNet() with the return values of the helper methods
# e.g. CreateDoubleArrayFromList(pU) takes a python list and returns a pointer to a 
# corresponding C array, that is passed as an argument to the method LoadNet()
mcf = MCFSimplex()
mcf.LoadNet(nmx, mmx, pn, pm, CreateDoubleArrayFromList(pU), CreateDoubleArrayFromList(pC),
            CreateDoubleArrayFromList(pDfct), CreateUIntArrayFromList(pSn),
            CreateUIntArrayFromList(pEn))

print( "Setting time..")
mcf.SetMCFTime()
mcf.SolveMCF()
#if mcf.MCFGetStatus() == 0:
   # print ("Optimal solution: %s" %mcf.MCFGetFO())
    #print( "Time elapsed: %s sec " %(mcf.TimeMCF()))
#else:
 #   print( "Problem unfeasible!")
  #  print( "Time elapsed: %s sec " %(mcf.TimeMCF()))


Setting time..


In [60]:
mcf.MCFGetFO()

1.7976931348623157e+308

In [11]:
#pip install numpy==1.6.1

In [180]:
vettore_soluzione={}

In [181]:
def showModuleFunctionality():
   
    nmx = mcf.MCFnmax()
    mmx = mcf.MCFmmax()
    pn = mcf.MCFnmax()
    pm = mcf.MCFmmax()

    pU = []
    caps = new_darray(mmx)
    mcf.MCFUCaps(caps)
    for i in range(0, mmx):
        pU.append(darray_get(caps, i))

    pC = []
    costs = new_darray(mmx)
    mcf.MCFCosts(costs)
    for i in range(0, mmx):
        pC.append(darray_get(costs, i))

    pDfct = []
    supply = new_darray(nmx)
    mcf.MCFDfcts(supply)
    for i in range(0, nmx):
        pDfct.append(darray_get(supply, i))

    pSn = []
    pEn = []
    startNodes = new_uiarray(mmx)
    endNodes = new_uiarray(mmx)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, mmx):
        pSn.append(uiarray_get(startNodes, i) + 1)
        pEn.append(uiarray_get(endNodes, i) + 1)

    print("arc flow")
    length = mcf.MCFm()
    flow = new_darray(length)
    length = mcf.MCFn()
    nms = new_uiarray(length)
    mcf.MCFGetX(flow, nms)

   

    for i in range(0, length):
        print("flow", darray_get(flow, i), "arc", uiarray_get(nms, i))
        vettore_soluzione[uiarray_get(nms, i)] = darray_get(flow, i)

    print("node potentials")
    length = mcf.MCFn()
    costs = new_darray(length)
    mcf.MCFGetPi(costs, nms)
    for i in range(0, length):
        print("flow", darray_get(costs, i), "node", i + 1)

    print("reading graph - arcs")
    length = mcf.MCFm()
    startNodes = new_uiarray(length)
    endNodes = new_uiarray(length)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, length):
        print("Arc %s: start %s end %s" % (i, uiarray_get(startNodes, i) + 1, uiarray_get(endNodes, i) + 1))

    print("reading graph - costs")
    length = mcf.MCFm()
    costs = new_darray(length)
    mcf.MCFCosts(costs)
    for i in range(0, length):
        print("Arc %s: cost %s" % (i, darray_get(costs, i)))

    print("reading graph - capacities")
    length = mcf.MCFm()
    caps = new_darray(length)
    mcf.MCFUCaps(caps)
    for i in range(0, length):
        print("Arc %s: capacities %s" % (i, darray_get(caps, i)))

    print("reading nodes - supply/demand")
    length = mcf.MCFn()
    supply = new_darray(length)
    mcf.MCFDfcts(supply)
    for i in range(0, length):
        print("Node %s: demand %s" % (i + 1, darray_get(supply, i)))

    return vettore_soluzione  # Restituisci il vettore_soluzione alla fine della funzione


In [182]:
#from pyMCFSimplex import *
#print( "pyMCFSimplex Version '%s' successfully imported." % version())
mcf = MCFSimplex()
#print ("MCFSimplex Class successfully instantiated.")
FILENAME = '1000/netgen-1000-1-1-a-a-ns.dmx'
#print ("Loading network from DIMACS file %s.." % FILENAME)
f = open(FILENAME,'r')
inputStr = f.read()
f.close()
mcf.LoadDMX(inputStr)
 
#print ("Setting time..")
#mcf.SetMCFTime()
#print ("Solving problem..")
mcf.SolveMCF()
if mcf.MCFGetStatus() == 0:
    print ("Optimal solution: %s" %mcf.MCFGetFO())
   
    print ("Time elapsed: %s sec " %(mcf.TimeMCF()))
else:
    print( "Problem unfeasible!")
    print ("Time elapsed: %s sec " %(mcf.TimeMCF()))

Optimal solution: 24633.0
Time elapsed: 0.0 sec 


In [192]:
%%capture
showModuleFunctionality()

In [ ]:
vettore_soluzione
sol_x=[0]*1000
for key in vettore_soluzione:
    if key<=1000:
    #print(key)
        sol_x[key-1]=vettore_soluzione[key]
print(sol_x)